## Import libraries

In [42]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

## Import files

In [43]:
top500 = pd.read_csv('../data/clean/DB_Kaggle_top500albums_clean.csv', encoding='latin1')
top500.head(3)

,toprankingorder,albumnamers,artistnamers,yearrs
0,1,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1967
1,2,Pet Sounds,The Beach Boys,1966
2,3,Revolver,The Beatles,1966


In [44]:
top500.rename(columns={'album': 'albumnamers'}, inplace=True)
top500.columns

Index(['toprankingorder', 'albumnamers', 'artistnamers', 'yearrs'], dtype='object')

In [45]:
spotifyalbums = pd.read_csv('../data/clean/api_albums_clean.csv', encoding='latin1')
spotifyalbums.head(3)

,albumnamers,albumidspotify,albumnamespotify,artistnamespotify,artistidspotify,releasedate,totaltracks
0,Sgt. Pepper's Lonely Hearts Club Band,6QaVfG1pHYl1z15ZxkvVDW,Sgt. Pepper's Lonely Hearts Club Band (Remaste...,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1967-06-01,13.0
1,Pet Sounds,6GphKx2QAPRoVGWE9D7ou8,Pet Sounds (Original Mono & Stereo Mix),The Beach Boys,3oDbviiivRWhXwIE8hxkVV,1966-06-16,27.0
2,Revolver,3PRoXYsngSwjEQWR5PsHWR,Revolver (Remastered),The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1966-08-05,14.0


## Merge tables

In [46]:
sqltable_top500 = pd.merge(top500, spotifyalbums[['albumnamers', 'albumidspotify']], how='left', on='albumnamers')

In [47]:
sqltable_top500.head(2)

,toprankingorder,albumnamers,artistnamers,yearrs,albumidspotify
0,1,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1967,6QaVfG1pHYl1z15ZxkvVDW
1,2,Pet Sounds,The Beach Boys,1966,6GphKx2QAPRoVGWE9D7ou8


In [48]:
sqltable_top500.shape

(508, 5)

## Clean NAs and duplicates

In [49]:
#clean NAs (some albums available in the RS DB were not found in Spotify)
sqltable_top500.dropna(inplace=True)

In [50]:
sqltable_top500.shape

(435, 5)

In [51]:
sqltable_top500.duplicated().any()

False

In [52]:
import re

# Iterate over each column in the DataFrame
for col in sqltable_top500.columns:
    sqltable_top500[col] = sqltable_top500[col].apply(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)

# Display the updated DataFrame
sqltable_top500.head()

,toprankingorder,albumnamers,artistnamers,yearrs,albumidspotify
0,1,Sgt Peppers Lonely Hearts Club Band,The Beatles,1967,6QaVfG1pHYl1z15ZxkvVDW
1,2,Pet Sounds,The Beach Boys,1966,6GphKx2QAPRoVGWE9D7ou8
2,3,Revolver,The Beatles,1966,3PRoXYsngSwjEQWR5PsHWR
3,4,Highway 61 Revisited,Bob Dylan,1965,6YabPKtZAjxwyWbuO9p4ZD
4,5,Rubber Soul,The Beatles,1965,50o7kf2wLwVmOTVYJOTplm


## Column names

In [53]:
sqltable_top500.columns

Index(['toprankingorder', 'albumnamers', 'artistnamers', 'yearrs',
       'albumidspotify'],
      dtype='object')

In [54]:
finalcolumns = ['rs_toprankingorder', 'rs_albumname', 'rs_artist', 'rs_year', 'sp_albumid']

In [55]:
sqltable_top500.columns = finalcolumns

In [56]:
sqltable_top500.head(1)

,rs_toprankingorder,rs_albumname,rs_artist,rs_year,sp_albumid
0,1,Sgt Peppers Lonely Hearts Club Band,The Beatles,1967,6QaVfG1pHYl1z15ZxkvVDW


## Reoder columns

In [57]:
#sqltable_top500 = sqltable_top500.iloc[:, [0, 1, 2, 3, 4,5]]

In [58]:
sqltable_top500['albums_sp_albumid'] = sqltable_top500['sp_albumid']

In [60]:
sqltable_top500.head(1)

,rs_toprankingorder,rs_albumname,rs_artist,rs_year,sp_albumid,albums_sp_albumid
0,1,Sgt Peppers Lonely Hearts Club Band,The Beatles,1967,6QaVfG1pHYl1z15ZxkvVDW,6QaVfG1pHYl1z15ZxkvVDW


## Data type

In [61]:
sqltable_top500.info()

<class 'pandas.core.frame.DataFrame'>
Index: 435 entries, 0 to 507
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   rs_toprankingorder  435 non-null    int64 
 1   rs_albumname        435 non-null    object
 2   rs_artist           435 non-null    object
 3   rs_year             435 non-null    int64 
 4   sp_albumid          435 non-null    object
 5   albums_sp_albumid   435 non-null    object
dtypes: int64(2), object(4)
memory usage: 23.8+ KB


In [62]:
sqltable_top500['rs_year'] = pd.to_datetime(sqltable_top500['rs_year'], format='%Y')

In [63]:
sqltable_top500['rs_year'].dtype

dtype('<M8[ns]')

## Export 

In [65]:
sqltable_top500.to_csv('../data/tableau/tableautable_top500.csv', index=True, encoding='latin1')